##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 템플릿을 사용하여 TFX 파이프라인 생성하기

Note: We recommend running this tutorial on Google Cloud Vertex AI Workbench. [Launch this notebook on Vertex AI Workbench](https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps%253A%252F%252Fraw.githubusercontent.com%252Ftensorflow%252Ftfx%252Fmaster%252Fdocs%252Ftutorials%252Ftfx%252Ftemplate.ipynb).

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/template"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/template.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a>
</td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/template.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table></div>

## 시작하기

이 문서에서는 TFX Python 패키지와 함께 제공되는 *템플릿*을 사용하여 TensorFlow Extended(TFX) 파이프라인 생성 지침을 제공합니다. 지침의 상당 부분은 AI Platform Notebooks 인스턴스에서 실행되는 Linux 쉘 명령어입니다. `!`를 사용하여 해당 명령을 호출하는 주피터 노트북 코드 셀이 제공됩니다.

시카고 시에서 출시된 [Taxi Trips 데이터세트](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)를 사용하여 파이프라인을 구축할 수 있습니다. 이 파이프라인을 활용하여 데이터세트를 사용하는 자체 파이프라인을 구축할 것을 권장합니다.


## 1단계. 환경 설정

AI Platform Pipeline은 개발 환경을 통해 파이프라인을 구축하고Kubeflow Pipeline 클러스터로 새로 빌드된 파이프라인을 실행합니다.

**참고:** 특정 TensorFlow 버전을 선택하거나 GPU 인스턴스를 선택하려면 AI Platform Notebooks에서 TensorFlow 사전 설치된 인스턴스를 생성하세요.


`kfp` 추가 요구 사항이 있는 `tfx` Python패키지를 설치합니다.

In [ ]:
import sys
# Use the latest version of pip.
!pip install --upgrade pip
# Install tfx and kfp Python packages.
!pip install --upgrade "tfx[kfp]<2"

TFX 버전을 확인합니다.

In [ ]:
!python3 -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"

AI Platform Pipelines에서 TFX는 [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/)을 사용하여 호스팅된 Kubernetes 환경에서 실행됩니다.

몇 가지 환경 변수를 설정하여 Kubeflow Pipelines를 사용해 보겠습니다.

우선 GCP 프로젝트 ID를 가져옵니다.

In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
print("GCP project ID:" + GOOGLE_CLOUD_PROJECT)

또한 KFP 클러스터에 액세스해야 합니다. 'AI 플랫폼&gt; 파이프라인' 메뉴의 Google Cloud Console에서 액세스할 수 있습니다. KFP 클러스터의 '엔드포인트'는 파이프라인 대시보드의 URL에서 찾거나 이 노트북을 시작한 시작하기 페이지의 URL에서 가져올 수 있습니다. `ENDPOINT` 환경 변수를 만들고 해당 환경 변수를 KFP 클러스터 엔드포인트로 설정해 보겠습니다. **엔드포인트는 URL의 호스트 이름 부분만 포함해야 합니다.** 예를 들어, KFP 대시보드의 URL이 `https://1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com/#/start`이면 엔드포인트 값은 `1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com`이 됩니다.

> **참고: 아래에서 엔드포인트 값을 설정해야 합니다.**

In [ ]:
# This refers to the KFP cluster endpoint
ENDPOINT='' # Enter your ENDPOINT here.
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

현재 GCP 프로젝트에서 이미지 이름을 `tfx-pipeline`으로 설정합니다.

In [ ]:
# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/tfx-pipeline'

이제 파이프라인을 생성할 수 있습니다.

## 2단계. 미리 정의된 템플릿을 프로젝트 디렉터리에 복사

이 단계에서는 미리 정의된 템플릿에서 추가 파일을 복사하여 작업 파이프라인 프로젝트 디렉터리 및 파일을 생성하겠습니다.

아래 `PIPELINE_NAME`을 변경하여 파이프라인에 다른 이름을 지정할 수 있으며, 이는 파일이 저장되는 프로젝트 디렉터리 이름으로도 사용됩니다.

In [ ]:
PIPELINE_NAME="my_pipeline"
import os
PROJECT_DIR=os.path.join(os.path.expanduser("~"),"imported",PIPELINE_NAME)

TFX에는 TFX Python 패키지가 있는 `taxi` 템플릿이 포함되어 있습니다. 분류 및 회귀 등의 포인트별 예측 문제를 해결하려는 경우 이 템플릿으로 시작할 수 있습니다.

`tfx template copy` CLI 명령은 미리 정의된 템플릿 파일을 프로젝트 디렉토리에 복사합니다.

In [ ]:
!tfx template copy \
  --pipeline-name={PIPELINE_NAME} \
  --destination-path={PROJECT_DIR} \
  --model=taxi

이 노트북의 작업 디렉토리 컨텍스트를 프로젝트 디렉토리로 변경합니다.

In [ ]:
%cd {PROJECT_DIR}

> 참고: 생성된 프로젝트 디렉토리를 클릭하여 왼쪽의 `File Browser`에서 디렉토리를 변경해야 합니다.

## 3단계. 복사한 소스 파일 검색

TFX 템플릿은 파이프라인의 출력을 분석하기 위해 Python 소스 코드, 샘플 데이터 및 주피터 노트북을 포함하여 파이프라인을 구축하기 위한 기본 스캐폴드 파일을 제공합니다. `taxi` 템플릿은 [Airflow 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/airflow_workshop)과 동일한 *Chicago Taxi* 데이터세트 및 ML 모델을 사용합니다.

다음은 각 Python 파일에 대한 간략한 소개입니다.

- `pipeline` -이 디렉토리는 파이프라인 정의를 포함합니다.
    - `configs.py` — 파이프라인 러너의 공통 상수를 정의합니다.
    - `pipeline.py` — TFX 컴포넌트 및 파이프라인을 정의합니다.
- `models` - 이 디렉토리는 ML 모델 정의를 포함합니다.
    - `features.py`, `features_test.py` — 모델의 특성을 정의합니다.
    - `preprocessing.py`, `preprocessing_test.py` — `tf::Transform`을 사용하여 전처리 작업을 정의합니다.
    - `estimator` - 이 디렉토리는 Estimator 기반 모델을 포함합니다.
        - `constants.py` — 모델 상수를 정의합니다.
        - `model.py`, `model_test.py` — TF estimator를 사용하여 DNN 모델을 정의합니다.
    - `keras` - 이 디렉토리는 Keras 기반 모델을 포함합니다.
        - `constants.py` — 모델의 상수를 정의합니다.
        - `model.py`, `model_test.py` — Keras를 사용하여 DNN 모델을 정의합니다.
- `local_runner.py`, `kubeflow_runner.py` — 각 오케스트레이션 엔진의 러너를 정의합니다.


이름에 `_test.py`가 포함된 일부 파일이 있음을 알 수 있습니다. 이는 파이프라인의 단위 테스트이며 자체 파이프라인을 구현할 때 더 많은 단위 테스트를 추가하는 것이 좋습니다. `-m` 플래그와 함께 테스트 파일의 모듈 이름을 제공하여 단위 테스트를 실행할 수 있습니다. 일반적으로 `.py` 확장자를 삭제하고 `/`를 `.`로 대체하여 모듈 이름을 얻을 수 있습니다. 예를 들면 다음과 같습니다.

In [ ]:
!{sys.executable} -m models.features_test
!{sys.executable} -m models.keras.model_test


## 4단계. 첫 번째 TFX 파이프라인 실행

TFX 파이프라인의 구성 요소는 각 실행에 대한 출력을 [ML 메타 데이터 아티팩트](https://www.tensorflow.org/tfx/guide/mlmd)로 생성하여 저장해야 합니다. KFP 클러스터가 액세스할 수 있는 저장소는 모두 사용 가능합니다. 다음 예제에서는 Google Cloud Storage(GCS)를 사용해 보겠습니다. 기본 GCS 버킷은 자동으로 생성되며, 이름은 `<your-project-id>-kubeflowpipelines-default`입니다.


나중에 파이프라인에서 사용할 수 있도록 샘플 데이터를 GCS 버킷에 업로드하겠습니다.

In [ ]:
!gsutil cp data/data.csv gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-template/data/taxi/data.csv

`tfx pipeline create` 명령을 사용하여 TFX 파이프라인을 생성해 보겠습니다.

> 참고: KFP용 파이프라인을 만들 때 파이프라인을 실행할 컨테이너 이미지가 필요하며, 실행 시 `skaffold`가 이미지를 구축할 것입니다. 스캐폴드는 도커 허브에서 기본 이미지를 가져오기 때문에 처음 이미지를 빌드할 경우에는 5~10분가량 걸리지만 그 이후에는 시간이 단축됩니다.

In [ ]:
!tfx pipeline create  --pipeline-path=kubeflow_runner.py --endpoint={ENDPOINT} \
--build-image

파이프라인을 생성하는 동안 Docker 이미지를 빌드하기 위해 `Dockerfile`이 생성되니, 다른 소스 파일과 함께 소스 제어 시스템(git 등)에 추가하기 바랍니다.

참고: `airflow`가 설치되지 않고 `--engine`이 지정되지 않는 경우 `kubeflow`가 자동으로 오케스트레이션 엔진으로 선택됩니다.

이제 `tfx run create` 명령을 사용하여 새로 생성된 파이프라인으로 실행을 시작합니다.

In [ ]:
!tfx run create --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

또는, KFP 대시보드에서 파이프라인을 실행할 수도 있습니다. 새로 실행할 경우 KFP 대시보드의 실험 아래에 나열됩니다. 실험을 클릭하면 진행 상황을 모니터링하고 실행 중에 생성된 아티팩트를 시각화할 수 있습니다.

그러나 KFP 대시보드를 이용할 것을 권장합니다. Google Cloud Console의 Cloud AI Platform Pipelines 메뉴에서 KFP 대시보드에 액세스할 수 있습니다. 대시보드에서는 파이프라인을 찾거나, 파이프라인에 대한 다양한 정보에 액세스할 수 있습니다. 예를 들어, *실험* 메뉴에서 실행을 찾을 수 있으며 실험 아래 실행을 열면 *아티팩트* 메뉴 아래 파이프라인에서 모든 아티팩트를 찾을 수 있습니다.

> 참고: 파이프라인이 실행에 실패하면 KFP 대시보드의 실험 탭에서 각 TFX 구성 요소에 대한 로그를 자세히 볼 수 있습니다.

실행이 실패하는 주요 원인 중 하나는 권한과 관련이 있습니다. KFP 클러스터에 Google Cloud API 액세스 권한이 있는지 확인하세요. 이 권한은 [GCP에서 KFP 클러스터를 생성할 때](https://cloud.google.com/ai-platform/pipelines/docs/setting-up) 구성할 수 있습니다. 또는 [GCP의 문제 해결 문서](https://cloud.google.com/ai-platform/pipelines/docs/troubleshooting)를 참조하세요.

## 5단계. 데이터 검증을 위한 구성 요소 추가

`StatisticsGen`, `SchemaGen`, `ExampleValidator`을 포함하여 데이터 검증을 위한 구성 요소를 추가합니다. 데이터 검증에 대해 자세히 알고 싶으면 [Tensorflow 데이터 검증 시작하기](https://www.tensorflow.org/tfx/data_validation/get_started)를 참조하세요.

> **두 번 클릭하여 디렉터리를 `pipeline`으로 변경하고 다시 두 번 클릭하여 `pipeline.py`**를 엽니다. `StatisticsGen`, `SchemaGen`, `ExampleValidator`를 파이프라인에 추가하는 세 줄을 찾아 주석 처리를 제거합니다. (팁: `TODO(step 5):`를 포함하는 주석을 찾습니다. 편집 후 `pipeline.py`를 저장해야 합니다.

이제 수정된 파이프라인 정의로 기존 파이프라인을 업데이트해야 합니다. `tfx pipeline update` 명령을 사용하여 파이프라인을 업데이트한 다음 `tfx run create` 명령을 사용하여 업데이트된 파이프라인의 새 실행을 생성합니다.


In [ ]:
# Update the pipeline
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
# You can run the pipeline the same way.
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

### 파이프라인 출력 확인하기

KFP 대시보드로 이동하여 파이프라인 실행 페이지에서 파이프라인 출력을 찾습니다. 왼쪽의 *실험* 탭을 클릭하고 실험 페이지에서 *모두 실행*을 선택합니다. 파이프라인 이름으로 최근 실행한 작업을 찾을 수 있습니다.

## 6단계. 훈련용 구성 요소 추가

이 단계에서는 `Transform`, `Trainer`, `Resolver`, `Evaluator`, `Pusher`를 포함하여 훈련 및 모델 검증을 위한 구성 요소를 추가합니다.

> **두 번 클릭하여 `pipeline.py`를 엽니다**. 파이프라인에 `Transform`, `Trainer`, `Resolver`, `Evaluator`, `Pusher`를 추가하는 5줄을 찾아 주석 처리를 제거합니다. (팁: `TODO(step 6):`을 찾음)

이전처럼 이제 수정된 파이프라인 정의로 기존 파이프라인을 업데이트해야 합니다. 지침은 5단계와 동일합니다. `tfx pipeline update`를 사용하여 파이프라인을 업데이트하고 `tfx run create`를 사용하여 실행을 생성합니다.


In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

이 실행이 성공적으로 완료되면 AI Platform Pipelines에서 TFX 파이프라인을 처음으로 생성하고 실행하게 됩니다.

**참고:** 모델 코드에서 변경한 사항이 있으면 컨테이너 이미지도 다시 빌드해야 합니다. `pipeline update` 명령의 `--build-image` 플래그를 사용하여 다시 빌드를 트리거할 수 있습니다.

**참고:** 파이프라인 실행을 생성할 때마다 입력 및 매개변수가 변경되지 않은 경우에도 모든 구성 요소가 계속해서 실행될 수 있습니다. 해당 실행은 파이프라인 캐싱으로 건너뛰어 시간과 리소스를 절약할 수 있습니다. `pipeline.py`의 `Pipeline` 객체에 `enable_cache=True`를 지정하여 캐싱을 활성화할 수 있습니다.


## 7단계. (*선택 사항*) BigQueryExampleGen 사용하기

[BigQuery](https://cloud.google.com/bigquery)는 확장성이 뛰어나며 비용 효율적인 서버리스 클라우드 데이터 웨어하우스입니다. BigQuery는 TFX에서 훈련 예제 소스로 사용할 수 있습니다. 이 단계에서는 `BigQueryExampleGen`를 파이프라인에 추가하겠습니다.

> **두 번 클릭하여 `pipeline.py`를 엽니다.** `CsvExampleGen`을 주석 처리하고 `BigQueryExampleGen`의 인스턴스를 생성하는 줄의 주석 처리를 제거합니다. 또한 `create_pipeline` 함수의 `query` 인수에 대한 주석 처리를 제거해야 합니다.

BigQuery에 사용할 GCP 프로젝트를 지정해야 하며, 이를 위해 파이프라인 생성 시 `beam_pipeline_args`에서 `--project`를 설정합니다.

> **두 번 클릭하여 `configs.py`를 엽니다.** `GOOGLE_CLOUD_REGION`, `BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS`, `BIG_QUERY_QUERY` 정의의 주석 처리를 제거합니다. 이 파일의 지역 값을 GCP 프로젝트의 올바른 값으로 교체해야 합니다.

> **참고: 더 진행하기 앞서 `configs.py` 파일에서 GCP 지역을 설정해야 합니다.**

> **디렉터리를 한 단계 위로 변경합니다.** 파일 목록 위의 디렉터리 이름을 클릭합니다. 디렉터리 이름을 변경하지 않았다면 `my_pipeline` 파이프라인 이름과 동일합니다.

> **두 번 클릭하여 `kubeflow_runner.py`를 엽니다.** `create_pipeline` 함수에서 `query` 및 `beam_pipeline_args`라는 두 인자의 주석 처리를 제거합니다.

이제 파이프라인에서 BigQuery를 예제 소스로 사용할 수 있습니다. 이전과 같이 파이프라인을 업데이트하고 5단계 및 6단계와 같이 새 실행을 생성합니다.

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

## 8단계. (*선택 사항*) KFP로 데이터 흐름 사용하기

여러 [TFX 구성 요소는 Apache Beam](https://www.tensorflow.org/tfx/guide/beam)을 사용하여 데이터 병렬 파이프라인을 구현하며, 이를 통해 [Google Cloud Dataflow](https://cloud.google.com/dataflow/)를 사용하여 데이터 처리 워크로드를 분산할 수 있습니다. 이 단계에서는 Kubeflow 오케스트레이터를 설정하여 데이터 흐름을 Apache Beam의 데이터 처리 백엔드로 사용하겠습니다.

> **`pipeline`을 두 번 클릭하여 디렉터리를 변경하고 두 번 클릭하여 `configs.py`를 엽니다.** `GOOGLE_CLOUD_REGION` 및 `DATAFLOW_BEAM_PIPELINE_ARGS` 정의에 대한 주석 처리를 제거합니다.

> **디렉터리를 한 단계 위로 변경합니다.** 파일 목록 위의 디렉터리 이름을 클릭합니다. 디렉터리 이름은 사전에 변경하지 않았다면 `my_pipeline`의 파이프라인 이름과 동일합니다.

> **두 번 클릭하여 `kubeflow_runner.py`를 엽니다.** `beam_pipeline_args` 주석 처리를 제거합니다. 또한 7단계에서 추가한 `beam_pipeline_args`를 주석 처리합니다.

이제 파이프라인에서 Dataflow를 사용할 수 있습니다. 파이프라인을 업데이트하고 5단계 및 6단계처럼 실행을 생성합니다.

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

[Cloud Console의 Dataflow](http://console.cloud.google.com/dataflow)에서 Dataflow 작업을 찾을 수 있습니다.


## 9단계. (*선택 사항*) KFP로 Cloud AI Platform 훈련 및 예측하기

TFX는 [훈련 및 예측을 위한 Cloud AI Platform](https://cloud.google.com/ai-platform/)과 같은 여러 관리형 GCP 서비스와 상호 운용됩니다. `Trainer` 구성 요소를 설정하여 ML 모델 학습을 위한 관리형 서비스인 Cloud AI Platform Training을 사용할 수 있습니다. 또한, 모델 구축 및 제공 시, 모델을 Cloud AI Platform Prediction으로 *푸시*할 수 있습니다. 이 단계에서는 Cloud AI Platform 서비스를 사용할 수 있도록 `Trainer` 및 `Pusher` 구성 요소를 설정하겠습니다.

> 파일을 수정하기 전에 먼저 *AI Platform Training &amp; Prediction API*를 사용 설정해야 할 수 있습니다.

> **`pipeline`을 두 번 클릭하여 디렉터리를 변경하고 두 번 클릭하여 `configs.py`를 엽니다.** `GOOGLE_CLOUD_REGION`, `GCP_AI_PLATFORM_TRAINING_ARGS`, `GCP_AI_PLATFORM_SERVING_ARGS`의 정의를 주석 해제합니다. 사용자 지정 빌드 컨테이너 이미지를 사용하여 Cloud AI Platform Training에서 모델을 학습하면 `GCP_AI_PLATFORM_TRAINING_ARGS`의 `masterConfig.imageUri`를 위의 `CUSTOM_TFX_IMAGE`와 동일한 값으로 설정할 수 있습니다.

> **디렉터리를 한 단계 위로 변경하고 두 번 클릭하여 `kubeflow_runner.py`를 엽니다.** `ai_platform_training_args` 및 `ai_platform_serving_args`의 주석 처리를 제거합니다.

파이프라인을 업데이트하고 5단계 및 6단계처럼 실행을 생성합니다.

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

[Cloud AI Platform Jobs](https://console.cloud.google.com/ai-platform/jobs)에서 훈련 작업을 찾을 수 있습니다. 파이프라인이 성공적으로 완료되면 [Cloud AI Platform Model](https://console.cloud.google.com/ai-platform/models)에서 모델을 찾을 수 있습니다.

## 10단계. 파이프라인에 데이터 수집

Chicago Taxi 데이터세트를 사용하여 모델의 파이프라인을 만들었습니다. 이제 데이터를 파이프라인에 입력하겠습니다.

데이터는 GCS 또는 BigQuery를 포함하여 파이프라인이 액세스할 수 있는 모든 곳에 저장할 수 있습니다. 파이프라인 정의를 수정해야 데이터에 액세스할 수 있습니다.

1. 데이터가 파일에 저장된 경우 `kubeflow_runner.py` 또는 `local_runner.py`에서 `DATA_PATH`를 수정하고 해당 파일 위치로 설정합니다. 데이터가 BigQuery에 저장되어 있는 경우, `pipeline/configs.py`의 `BIG_QUERY_QUERY`를 수정하여 데이터를 올바로 쿼리할 수 있습니다.
2. `models/features.py`에서 특성을 추가합니다.
3. `models/preprocessing.py`를 수정하여 [훈련에 적합하게 입력 데이터로 변환](https://www.tensorflow.org/tfx/guide/transform)합니다.
4. `models/keras/model.py` 및 `models/keras/constants.py`를 수정하여 [ML 모델을 설명](https://www.tensorflow.org/tfx/guide/trainer)합니다.
    - Estimator 기반 모델도 사용할 수 있습니다. `pipeline/configs.py`에서 `RUN_FN` 상수를 `models.estimator.model.run_fn`으로 변경합니다.

자세한 소개는 [Trainer 구성 요소 가이드](https://www.tensorflow.org/tfx/guide/trainer)를 참조하세요.

## 정리

이 프로젝트에서 사용한 모든 Google Cloud 리소스를 정리하려면 본 튜토리얼에서 사용한 [Google Cloud 프로젝트를 삭제](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)하면 됩니다.

또는 아래의 각 콘솔로 이동하여 개별 리소스를 정리할 수 있습니다.

- [Google Cloud Storage](https://console.cloud.google.com/storage)
- [Google Container Registry](https://console.cloud.google.com/gcr)
- [Google Kubernetes Engine](https://console.cloud.google.com/kubernetes)
